In [17]:
import gym
from gym import spaces
import pygame
import numpy as np
from panel import state


def allboard(size):
    board = []
    for x in range(size):
        for y in range(size):
            board.append(np.array([x,y]))
    return np.array(board)

class GridWorldEnv(gym.Env):
    metadata = {"render_modes": ["human", "rgb_array"], "render_fps": 4}

    def __init__(self, render_mode=None, size=5, num_agents = 2):
        self.size = size  # The size of the square grid
        self.num_agents = num_agents  # The number of agents
        self.window_size = 512  # The size of the PyGame window

        self.board = allboard(size)
        # Observations are dictionaries with the agent's and the target's location.
        # Each location is encoded as an element of {0, ..., `size`}^2, i.e. MultiDiscrete([size, size]).
        self.observation_space = spaces.Dict(
            {
                "agent": spaces.Box(0, size - 1, shape=(2,), dtype=int),
                "target": spaces.Box(0, size - 1, shape=(2,), dtype=int),
            }
        )
        self._target_location = np.array([0,self.size**2-1])

        # We have 4 actions, corresponding to "right", "up", "left", "down"
        self.action_space = spaces.Discrete(5)

        """
        The following dictionary maps abstract actions from `self.action_space` to 
        the direction we will walk in if that action is taken.
        I.e. 0 corresponds to "right", 1 to "up" etc.
        self._action_to_direction = {
            0: np.array([1, 0]),
            1: np.array([0, 1]),
            2: np.array([-1, 0]),
            3: np.array([0, -1]),
            4: np.array([0, 0]),
        }
        """
        self._action_to_direction = np.array([ -size, size, 1, -1, 0])
        '''{
            0: 1,
            1: -size,
            2: -1,
            3: size,
            4: 0,
        }'''

        assert render_mode is None or render_mode in self.metadata["render_modes"]
        self.render_mode = render_mode

        """
        If human-rendering is used, `self.window` will be a reference
        to the window that we draw to. `self.clock` will be a clock that is used
        to ensure that the environment is rendered at the correct framerate in
        human-mode. They will remain `None` until human-mode is used for the
        first time.
        """
        self.window = None
        self.clock = None

        self.actions, self.action_converter = self.getactions(5, num_agents)
        self.states, self.state_converter = self.getstates(size, num_agents)
        self.nextStates = self.getNextStates(self.states, self.state_converter)
        
    def _get_obs(self):
        return {"agent": self._agent_location, "target": self._target_location}
    
    def _get_info(self):
        return -1#{"distance": np.linalg.norm(self._agent_location - self._target_location, ord=1)}
    
    def reset(self, seed=None, options=None, start = None):
        #self.close()
        # We need the following line to seed self.np_random
        super().reset(seed=seed)

        # Choose the agent's location uniformly at random
        if start == None:
            agent_locations = []
            possible_loc = 0
            while possible_loc in self._target_location:
                possible_loc = self.np_random.integers(0, self.size**2, size=None, dtype=int)
            agent_locations.append(possible_loc)
            for x in range(self.num_agents-1):
                while possible_loc in self._target_location or possible_loc in agent_locations:
                    possible_loc = self.np_random.integers(0, self.size**2, size=None, dtype=int)
                agent_locations.append(possible_loc)
        
            self._agent_location = np.array(agent_locations)#self.np_random.integers(0, self.size, size=2, dtype=int)
        else:
            self._agent_location = self.states[start].copy()
        

        # We will sample the target's location randomly until it does not coincide with the agent's location

        observation = self._get_obs()
        info = self._get_info()

        if self.render_mode == "human":
            self._render_frame()

        return observation, info
    
    def step(self, action):
        terminated = True
        
        realact = np.sum(action*self.action_converter)
        realstate = np.sum(self._agent_location*self.state_converter)
        
        self._agent_location = self.nextStates[realstate][realact]

        for x in self._agent_location:
            if x not in self._target_location:
                terminated = False
                break

        reward = 1 if terminated else 0  # Binary sparse rewards
        observation = self._get_obs()
        info = self._get_info()

        if self.render_mode == "human":
            self._render_frame()

        return observation, reward, terminated, False, info
    
    def render(self):
        if self.render_mode == "rgb_array":
            return self._render_frame()

    def _render_frame(self):
        if self.window is None and self.render_mode == "human":
            pygame.init()
            pygame.display.init()
            self.window = pygame.display.set_mode((self.window_size, self.window_size))
        if self.clock is None and self.render_mode == "human":
            self.clock = pygame.time.Clock()

        canvas = pygame.Surface((self.window_size, self.window_size))
        canvas.fill((255, 255, 255))
        pix_square_size = (
            self.window_size / self.size
        )  # The size of a single grid square in pixels

        # First we draw the target
        for loc in self._target_location:
            location = self.board[loc]
            pygame.draw.rect(
                canvas,
                (255, 0, 0),
                pygame.Rect(
                    pix_square_size * location,
                    (pix_square_size, pix_square_size),
                ),
            )
        # Now we draw the agent
        for loc in self._agent_location:
            location = self.board[loc]
            pygame.draw.circle(
                canvas,
                (0, 0, 255),
                (location + 0.5) * pix_square_size,
                pix_square_size / 3,
            )

        # Finally, add some gridlines
        for x in range(self.size + 1):
            pygame.draw.line(
                canvas,
                0,
                (0, pix_square_size * x),
                (self.window_size, pix_square_size * x),
                width=3,
            )
            pygame.draw.line(
                canvas,
                0,
                (pix_square_size * x, 0),
                (pix_square_size * x, self.window_size),
                width=3,
            )

        if self.render_mode == "human":
            # The following line copies our drawings from `canvas` to the visible window
            self.window.blit(canvas, canvas.get_rect())
            pygame.event.pump()
            pygame.display.update()

            # We need to ensure that human-rendering occurs at the predefined framerate.
            # The following line will automatically add a delay to keep the framerate stable.
            self.clock.tick(self.metadata["render_fps"])
        else:  # rgb_array
            return np.transpose(
                np.array(pygame.surfarray.pixels3d(canvas)), axes=(1, 0, 2)
            )
        
    def close(self):
        if self.window is not None:
            pygame.display.quit()
            pygame.quit()


    def getstates(self, size, num_agents):
        states = []
        robvis = []
        for j in range(num_agents):
            robvis.append((size*size)**(num_agents-1-j))

        for i in range((size*size)**num_agents):
            robots = []
            work = i
            for r in robvis:
                #if work//r not in goals:
                robots.append(work//r)
                work = work - (work//r)*r
            states.append(np.array(robots))
        return np.array(states), np.array(robvis)


    def getactions(self, num_acts, num_agents):
        states = []
        robvis = []
        for j in range(num_agents):
            robvis.append((num_acts)**(num_agents-1-j))

        for i in range((num_acts)**num_agents):
            robots = []
            work = i
            for r in robvis:
                #if work//r not in goals:
                robots.append(work//r)
                work = work - (work//r)*r
            states.append(np.array(robots))
        return np.array(states), np.array(robvis)

    #returns the next state given an action
    def getNextStates(self, allstates, state_converter):
        size = self.size
        num_agents = self.num_agents
        num_acts = len(self._action_to_direction)
        
        allnexts = []
        for i in range((size*size)**num_agents):
            smallnext =[]
            actionstates = allstates[i]
            for j in range(len(self._action_to_direction) **num_agents):
                _agent_location = allstates[i].copy()
                action = self.actions[j]
                

                future_loc = []
                #for x in range(self.num_agents):
                # Map the action (element of {0,1,2,3}) to the direction we walk in
                direction = self._action_to_direction[action]
                # We use `np.clip` to make sure we don't leave the grid
                stateNext = _agent_location + direction
                stateNext = np.where(stateNext >= size*size,      _agent_location, stateNext)
                stateNext = np.where(stateNext < 0,               _agent_location, stateNext)
                
                for x in range(num_agents):
                    if direction[x] == 1 and _agent_location[x]%size == size -1:
                        stateNext[x] = _agent_location[x].copy()
                    elif direction[x] == -1 and _agent_location[x]%size == 0:
                        stateNext[x] = _agent_location[x].copy()


                for x in range(num_agents):
                    if stateNext[x] not in self._target_location:
                        if stateNext[x] in stateNext[0:x] or stateNext[x] in stateNext[x+1:num_agents]:
                            stateNext[x] = _agent_location[x].copy()
                            
                
                smallnext.append(stateNext)
            allnexts.append(smallnext)
        return np.array(allnexts)

In [24]:


def hasNoDuplicates(listOfElems):
    ''' Check if given list contains any duplicates '''
    if len(listOfElems) == len(set(listOfElems)):
        return True
    else:
        return False



def evaluate(policy, values, size, num_robots, actions,env):
    gamma = 1/2#((size*size)**num_robots)
    reward = -1#/(len(actions)**num_robots)
    #print(reward, gamma)
    
    nextValues = np.zeros(np.shape(values))
    #goals = [0, (size*size)**num_robots-1, size*size-1, (size*size)**num_robots-1- size*size-1] #[0,0], [0,8], [8,0], [8,8]
    smallgoals = [0, size*size-1]
    
        
    for i in range(1,(size*size)**num_robots):
        state = env.states[i]
        stateval = 0
        for k in range(len(actions)**num_robots):
            stateNext = env.nextStates[i][k]
            #print("s", env.nextStates[i])
            encodedNext = np.sum(stateNext * env.state_converter)
            stateval += policy[i][k] * (reward + gamma * values[encodedNext])
            #print("stateval",stateval)
        final = True
        for x in state:
            if x not in smallgoals:
                final = False
                break
        if not final:
            nextValues[i] = stateval
                #        stateval += policy[state[0]][state[1]][action] * (reward + gamma * values[stateNext[0]][stateNext[1]])
                #    nextValues[state[0]][state[1]] = stateval
    return nextValues
        

        
def iterate(policy, values, size, num_robots, actions,env):
    gamma = 1#((size*size)**num_robots)
    reward = -1#/(len(actions)**num_robots)
    #print(reward, gamma)
    
    qactc = np.zeros(np.shape(policy))
    #goals = [0, (size*size)**num_robots-1, size*size-1, (size*size)**num_robots-1- size*size-1] #[0,0], [0,8], [8,0], [8,8]
    smallgoals = [0, size*size-1]
    
        
    for i in range((size*size)**num_robots):
        
        allNext = env.nextStates[i]
        encodedNext = values[np.sum(allNext * env.state_converter, axis = 1)]
        #print(encodedNext)
        
        maxval = encodedNext[np.argmax(encodedNext)]
        #print("duh", state, maxval)
        #print(allNext)
        count = (encodedNext == maxval).sum()
        for y in range(len(actions)**num_robots):
            if encodedNext[y] == maxval:
                qactc[i][y] = 1/count
            else:
                qactc[i][y] = 0
                
    return qactc

def knownGridworld(size, num_robots):
    env = GridWorldEnv(render_mode = "rgb_array",size = size, num_agents = num_robots)

    actions = np.array([ -size, size, 1, -1, 0])
    if (size*size)**(num_robots)*(len(actions)**num_robots) > 100000000:
        print("Too complex")
        return
    print("max array size: ", (size*size)**(num_robots)*(len(actions)**num_robots))
    policy = np.ones(((size* size)** num_robots, len(actions)**num_robots))/len(actions)**num_robots
    
    values = np.zeros((size* size)** num_robots)
    nextvals = evaluate(policy, values, size, num_robots, actions, env)
    
    for y in range (10):
        for x in range(100):
            values = nextvals.copy()
            #print(nextvals)
            nextvals = evaluate(policy, values, size, num_robots, actions, env)
            delta = np.sum(np.abs(nextvals - values))
            print("eval: ",delta)
            if delta <.001:
                break
        
        newpolicy = iterate(policy, nextvals, size, num_robots, actions, env)
        delpol = np.sum(np.abs(newpolicy - policy))
        print("iter: ",delpol)
        if delpol < .1:
            break
        policy = newpolicy
    
    env.close()
    return newpolicy

def unknownGridworld(size, num_agents):
    env = GridWorldEnv(render_mode = "rgb_array",size = size, num_agents = num_agents)

    actions = np.zeros(((size* size)** num_agents, len(env._action_to_direction) **num_agents))

    epsilon = .8
    discount = .8
    #training 
    for i in range(5000):
        for x in range((size*size)**num_agents):
            #state = env.states[x]
            obs, info = env.reset(start = x)
            episode = []
            
            ended = False
            for x in range(10):
                truestate = np.sum(env.state_converter*obs["agent"])
                if np.random.random() > epsilon:
                    randact = np.random.randint(0, len(env._action_to_direction)**num_agents)
                else:
                    randact = np.argmax(actions[truestate])
                action = env.actions[randact]
                episode.append([truestate, randact])
                #env.clock(1)
                obs, reward, terminated, boom, info = env.step(action)
                if terminated:
                    ended = True
                    break
                
            y = -10
            for x in range(len(episode)):
                if ended:
                    y = -1*(len(episode) - x - 1)
                state = episode[x][0]
                action = episode[x][1]
                actions[state][action] = (actions[state][action]*discount + y * (1-discount))

            #print(episode)

    env.close()
    return actions

In [28]:
size = 3
num_agents = 3
firstpol = knownGridworld(size, num_agents)
secondpol = unknownGridworld(size, num_agents)


max array size:  91125
eval:  357.36400000000026
eval:  177.19963199999992
eval:  87.8469415039995
eval:  43.544720142591835
eval:  21.582546767037158
eval:  10.696476475523422
eval:  5.301020078884821
eval:  2.627034800583698
eval:  1.3018607796048793
eval:  0.6451468488049734
eval:  0.31970525977147557
eval:  0.15843067374767927
eval:  0.07851049586099523
eval:  0.038905906714391536
eval:  0.019279820031169237
eval:  0.009554108544324258
eval:  0.004734533749106706
eval:  0.0023461953385903733
eval:  0.001162655519216127
eval:  0.0005761531070678227
iter:  1419.1520000000005
eval:  214.1024619253078
eval:  237.18604296134112
eval:  0.4540803127661306
eval:  0.0
iter:  964.0045351473922
eval:  9.636735853746359e-14
iter:  0.0


In [29]:

#environment is known, and calculates it based on every state

env = GridWorldEnv(render_mode = "human", size = size, num_agents = num_agents)
for x in range(0, (size*size)**num_agents):
    #state = env.states[x]
    observation, info = env.reset(start = x)
    env.clock.tick(2)
    episode = [x]
    
    
    for x in range(0,100):
        truestate = np.sum(env.state_converter*observation["agent"])
        randact = np.argmax(firstpol[truestate])
        action = env.actions[randact]
        #env.clock(1)
        
        observation, reward, terminated, boom, info = env.step(action)
        if terminated:
            break
        episode.append(np.sum(env.state_converter*observation["agent"]))
    if len(episode) >= 3:
        print(episode)
        
env.close()


KeyboardInterrupt: 

In [27]:


#environment is unknown, and calculates it based on episodes starting at each state

env = GridWorldEnv(render_mode = "human", size = size, num_agents = num_agents)
for x in range(0, (size*size)**num_agents):
    #state = env.states[x]
    observation, info = env.reset(start = x)
    env.clock.tick(2)
    episode = [x]
    
    
    for x in range(0,100):
        truestate = np.sum(env.state_converter*observation["agent"])
        randact = np.argmax(secondpol[truestate])
        action = env.actions[randact]
        #env.clock(1)
        
        observation, reward, terminated, boom, info = env.step(action)
        if terminated:
            break
        episode.append(np.sum(env.state_converter*observation["agent"]))
    if len(episode) >= 3:
        print(episode)
        
env.close()


KeyboardInterrupt: 

In [30]:
env.close()